# Structural Optimization of a wingbox

In the previous notebook we have seen the structural analysis of a single wingbox cell. We would like however to use these tools to optimize the various parameters of the wingbox in order to minimize the weight of the structure. In order to do so let's remind ourselves what exactly we can change in order to improve the performance of this wingbox.

In the previous notebook we had defined the following parameters in order to perform the analysis.

- $n_{cell}$    - Number of cells
- $\text{loc}_{sp}$    - The location of the spars
- $t_{sk}$      - The thickness of the skin in each cell
- $A_{str}$     - The area of the stringers
- $t_{sp}$      -  The thickness of the spars
- $n_{str}$     - The amount of stringers per cell


These variables were then loaded in the API data structure as shown below:

```C
attr_dict = {
    "n_cell":4,
    "spar_loc_nondim":[0.3, 0.5, 0.7],
    "t_sk_cell":[0.002,0.004,0.004, 0.004],
    "area_str":20e-6,
    "t_sp":0.008,
    "str_cell":[8,8,7,8]
}

wingbox_struct = tud.Wingbox(**attr_dict)
```

The parameters which we will optimize for are the skin thickness in each cell, the area of the stringers, the thickness of the spars and finally the stringer of the cell. This leaves the amount of cells, the spar locations and the bay length to be decided by the designers. The reasoning behind this is that usually your rib and spar locations are constrained the  placement of other systems in the wing such as the flap and slat mechanism. Below a summary can be found of the fixed and optimization parameters:


| **Optimiziation Parameters**         	| **Fixed Parameters** 	|
|--------------------------------------	|----------------------	|
| Skin thickness in each cell          	| The amount of cells  	|
| Stringer area                        	| Spar locations       	|
| Spar thickness                       	| Bay length           	|
| The amount of stringers in each cell 	|                      	|



# Constraining the design

During the optimization it is important we set the minimum constraints the design should meet. Otherwise, the optimum solution would be no wingbox at all. We will look at several constraints, let us start with the usually more critical constraints.

### Buckling

To define the buckling constraints we will divide the wing in sections called bays. Each bay is enclosed by two ribs, thus the length of these bays is for the designer to decide. Once, a bay is defined, we can further split this up in a collection of flat sheets. Where each sheet is in turn enclosed by stringers. The boundary conditions for these stringers have been decided to be simply supported.

<br>
<p align="center"  id="figure1" >
<img src="https://raw.githubusercontent.com/saullocastro/tuduam/93cd6ee8368661ba99f1f0b222a4535df1994bab/notebooks/static/naca4412-il.svg" width="400" /> 
 <br>
</p>
<p align="center">
<em>Figure 1: The NACA 4412 airfoil</em>
</p>
<br>




